In [ ]:
from collections import defaultdict
from cronjob import get_repositories, get_repository_state, update_repository
from crowdin import delete_code_translations, delete_translation, get_orphaned_files_as_data_frame
from file_manager import get_crowdin_file, get_local_file
import git
import json
import os
import pandas as pd
import re
from zendesk import update_zendesk_articles, zendesk_request

In [ ]:
repositories_df = pd.read_csv('repositories.csv', comment='#')
repositories_df

In [ ]:
def get_update_result(index):
    repository = get_repositories(False)[index]
    return repository, get_repository_state(repository)

# Update ZenDesk articles

In [ ]:
repository = get_repositories(False)[0]
assert(repository.github.origin == 'holatuwol/zendesk-articles')

In [ ]:
uat_domain = 'liferaysupport1528999723.zendesk.com'
prod_domain = 'liferay-support.zendesk.com'

In [ ]:
zendesk_articles = update_zendesk_articles(repository, uat_domain)

# Refresh repository

In [ ]:
repository, update_result = get_update_result(0)
assert(repository.github.origin == 'holatuwol/zendesk-articles')

In [ ]:
new_files, all_files, file_info = update_result

In [ ]:
update_result = update_repository(repository)

In [ ]:
from scrape_liferay import authenticate, session

In [ ]:
all_files

In [ ]:
def add_zendesk_translation(file, title):
    disclaimer_text = '''
<p class="alert alert-info"><span class="wysiwyg-color-blue120">
ファストトラック記事は、お客様の利便性のために一部機械翻訳されています。また、ドキュメントは頻繁に更新が加えられており、翻訳は未完成の部分が含まれることをご了承ください。最新情報は都度公開されておりますため、必ず英語版をご参照ください。翻訳に問題がある場合は、[こちら|support-content-jp@liferay.com]までご連絡ください。
</span></p>
'''
    
    target_file = 'ja/' + file[3:] if file[0:3] == 'en/' else file.replace('/en/', '/ja/')

    if git.ls_files(target_file) != '' and git.diff(target_file) == '':
        print('%s already translated' % file)
        return

    with open(target_file, 'r') as f:
        new_file_content = disclaimer_text + f.read()

        x = target_file.rfind('/') + 1
        y = target_file.find('-', x)
        article_id = target_file[x:y]
        print(article_id)

        json_params = {
            'translation': {
                'locale': 'ja',
                'title': title,
                'body': new_file_content                
            }
        }
        
        domain = uat_domain
        api_path = '/help_center/articles/%s/translations.json' % article_id

        parameters = {}
        url = 'https://%s/api/v2%s' % (domain, api_path)

        r = session.post(url, json=json_params)

        print(r.status_code, r.text)

In [ ]:
os.chdir(repository.github.git_root)

for file in all_files:
    add_zendesk_translation(file, zendesk_articles['360024413271']['title'])

# Remove code blocks

In [ ]:
repository, update_result = get_update_result(1)
assert(repository.github.origin == 'holatuwol/liferay-docs')

In [ ]:
new_files, all_files, file_info = update_result

In [ ]:
delete_code_translations(
    repository,
    'en/develop/tutorials/articles/01-introduction-to-liferay-development/00-intro.markdown',
    file_info)

In [ ]:
for file in all_files:
    delete_code_translations(repository, file, file_info)

# Delete files from project

In [ ]:
repository, update_result = get_update_result(0)
assert(repository.github.origin == 'holatuwol/zendesk-articles')

In [ ]:
new_files, all_files, file_info = update_result

In [ ]:
for key in [key for key, value in file_info.items() if key.find(repository.crowdin.dest_folder) == 0 and 'id' in value]:
    print(key)
    #delete_translation(repository, key)

# Check orphaned files statistics

In [ ]:
repository, update_result = get_update_result(3)
assert(repository.github.origin == 'holatuwol/liferay-learn')

In [ ]:
df = get_orphaned_files_as_data_frame(repository, update_result)

In [ ]:
def get_crowdin_link(file_id):
    return '<a href="https://crowdin.com/translate/liferay-documentation/%s/en-ja">%s</a>' % (file_id, file_id)

In [ ]:
df.style.format({'id1': get_crowdin_link, 'id2': get_crowdin_link})

In [ ]:
df[df['translated1'] > 0].style.format({'id1': get_crowdin_link, 'id2': get_crowdin_link})